<h1>Using Multi Agent system for creating React app</h1>

In [1]:
# Import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [ ]:

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import json
import asyncio
from IPython.display import display, Markdown

ModuleNotFoundError: No module named 'agent_instructions'

In [ ]:
ideator_instructions = """ 
You are an expert Software developer. You're solving the next big user problems
for small business. Discover the next big pain points for small businesses. Come up with a list of 
5 innovative ideas that can be solved with React app, using Agentic AI. Rate each idea based on 
its potential impact on solving user problems and its feasibility for development. Respond 
with only a JSON, each containing the following fields:
- idea: A short description of the idea. 
- impact: A rating from 1-10 on solving user problems.
- cost: A rating from 1-10 on the cost for user to use the app.
"""

In [ ]:
# This Agent comes up with innovative ideas,to solve problems for small business.
thinker_agent = Agent(
    name="Ideator",
    instructions=ideator_instructions,
    model="gpt-4o",
    handoff_description="Select the best idea from the list"
)

In [ ]:
# Idea Selector Agent selects the best idea from the list, based on solving user's problems and cost considerations.
idea_selector = Agent(
    name="Idea Selector",
    instructions=f"You select the best idea from {thinker_agent} list, based on solving user problems and cost to user.",
    model="gpt-4o-mini",
    handoffs=
)


In [ ]:
app_file_structure_instructions = f"""
Create a detailed react app file structure for the idea selected by the Idea Selector agent.
Respond with only a JSON, each containing the following fields:
- directory: The directory in the project.
    - filename: The name of the file in the project.
    - file_type: The type of the file (e.g., .tsx, .js, .css, .json, etc.).
        - content: empty string for now.
    - purpose: The purpose of the file in the project.
-components: All component files required for the project.
-pages: All page files required for the project.
-services: All service files required for the project.
-utils: All utility files required for the project.
-hooks: All custom hook files required for the project.
-database: The database schema used in the project.
    -schema: User schema and other necessary schemas.
-API: The API used in the project.
-Payment: The payment gateway used in the project, such as Stripe or PayPal.
    - Monthly subscription or one-time payment.
-authentication: The authentication method used in the project, such as OAuth.
    - Social login options like Google, Facebook, etc.
-dependencies: The dependencies required for the file.
Make sure to include all necessary files and directories for a complete React application

-Crucial:
-Please include files package.json, tsconfig.json, .gitignore, public/index.html, src/index.tsx, and src/App.tsx.
-Double check for any missing files or directories. 
-Double check all of the structures and files are included.
"""

file_structure_agent = Agent(
    name="File Structure Creator",
    instructions=app_file_structure_instructions,
    model="gpt-4o-mini",
)


In [ ]:
developer_instructions = """ 
You are an expert React developer. You will be provided with a file structure from File Structure Creator Agent,  for a React app.
Your task is to generate the code for each file in the structure. You are provided with 
the tools to create files and write code to them.
-Crucial:
Please use tools to write the code to files. 
Divide the code, so each Agent can write 10% of the code from the file structure. 
Your job is to check the code written by the other Agents, and make sure the code is correct. 
If you find any issues with the code, fix them using the tools provided.
You can also add comments to the code, to explain what the code is doing.
"""

In [ ]:
developer_agent = Agent(
    name="Code developer",
    instructions=developer_instructions,
    model="gpt-4o-mini",
)    

In [ ]:
message = "Create a React app to solve the problems of small business."
with trace('Parallel Multi-Agent React App Creator'):
    results = await asyncio.gather(
        Runner.run(thinker_agent,'Come up with innovative ideas for small business.'),
        Runner.run(idea_selector,'Select the best idea from the list provided by the thinker agent.'),
        Runner.run(file_structure_agent,'Create a detailed react app file structure for the selected idea.'),
        Runner.run(developer_agent,'Write code for the login page of the app.'),
    )
    
    outputs = [result.final_output for result in results]
    
    for output in outputs:
        print(output)

Second version | 
Hierarchical Agents 

In [ ]:
thinker_instruction = """Please come up with innovative ideas for small business, using Agentic AI.
The ideas should focus on solving real world problems, with low cost to the user.
Respond with only a JSON, each containing the following fields:
- idea: A short description of the idea.
- impact: A rating from 1-10 on solving user problems.
- cost: A rating from 1-10 on the cost for user to use the app."""

In [ ]:

idea_selector_instruction = "Please select the best idea from the list provided by the thinker agent."


In [ ]:

app_file_structure_instructions = """Please create a detailed react app file structure for the idea selected by the Idea Selector agent. 
The file structure should include all necessary files and directories for a complete React applications.
Also include files package.json, tsconfig.json, .gitignore, public/index.html, src/index.tsx, and src/App.tsx.
Respond with only a JSON, each containing the following fields:
- directory: The directory in the project.
    - filename: The name of the file in the project.
    - file_type: The type of the file (e.g., .tsx, .js, .css, .json, etc.).
        - content: empty string for now.
    - purpose: The purpose of the file in the project.
-components: All component files required for the project.
-pages: All page files required for the project.
    * PageName.tsx: The main file for the page
-services: All service files required for the project.
-utils: All utility files required for the project.
-hooks: All custom hook files required for the project.
-database: The database schema used in the project.
    -schema: User schema and other necessary schemas.
-API: The API used in the project.
-Payment: The payment gateway used in the project, such as Stripe or PayPal.
    - Monthly subscription or one-time payment.
-authentication: The authentication method used in the project, such as OAuth.
    - Social login options like Google, Facebook, etc.
-dependencies: The dependencies required for the file.
Make sure to include all necessary files and directories for a complete React application.
Double check for any missing files or directories. 
Double check all of the structures and files are included. """


In [ ]:
developer_instruction = """Please write the code for the app file structure provided by the File Structure Creator agent.
Create as many agents as needed to write the code for the entire file structure.
Divide the code, so each Agent can write 10% of the code from the file structure.
Your job is to check the code written by the other Agents, and make sure the code is correct.
If you find any issues with the code, fix them.
Respond with JSON each containing the following fields:
AgentName: The name of the agent writing the code.
FileName: The name of the file being written.
DirectoryPath: The path of the directory where the file is will be saved.
FileType: The type of the file being written.
Code: The code written by the agent."""
 

In [ ]:

developer_agent = Agent(
    name="Directory Creator",
    instructions=app_file_structure_instructions,
    model="gpt-4o-mini",
)
streamed= ""
results = Runner.run_streamed(developer_agent, developer_instruction)
async for event in results.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        streamed += event.data.delta
        #print(event.data.delta, end='', flush=True)

In [ ]:
display(Markdown(streamed))

In [ ]:
data = json.loads(streamed)
print(data)